In [1]:
from google.colab import drive
drive.mount('/content/drive')  #listになかったのでドライブ上にデータ保存し、ドライブをマウント。

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.ensemble import RandomForestClassifier

In [3]:
# データの読み込み
feat = "feat02"
train = pd.read_csv(f"/content/drive/MyDrive/signate/債務不履行/途中経過_csv/feat_train_{feat}.csv")
test = pd.read_csv(f"/content/drive/MyDrive/signate/債務不履行/途中経過_csv/feat_test_{feat}.csv")

In [4]:
## カテゴリ変数の抽出
cols_notcat = ['MIS_Status', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
               'DisbursementGross', 'GrAppv', 'SBA_Appv','population', 'density',
               'lr_avg_ppi','ApprovalToDisbursement_days','outlier'] # 数値特徴量 'DisbursementGross_minus_SBA_Appv'
# county_col = ['county_gdp','sector_count']
time_col = ["DisbursementDate_timestamp",	"ApprovalDate_timestamp",	"LoanEnd_timestamp"]
dissba = ['DisbursementGross_plus_SBA_Appv','DisbursementGross_minus_SBA_Appv', 'DisbursementGross_times_SBA_Appv','DisbursementGross_divided_by_SBA_Appv']
sbagr = ['SBA_Appv_plus_GrAppv','SBA_Appv_minus_GrAppv', 'SBA_Appv_times_GrAppv','SBA_Appv_divided_by_GrAppv']
disgr = ['DisbursementGross_plus_GrAppv','DisbursementGross_minus_GrAppv', 'DisbursementGross_times_GrAppv','DisbursementGross_divided_by_GrAppv']
# year_col = ["USREC_SUM"]
# urb_col = ["HOUPCT_RUR","ALAND_PCT_RUR","RUR_pct"]
city_col = ["city_Term_mean","city_NoEmp_mean"]

cols_notcat = cols_notcat + city_col + dissba + sbagr + disgr + time_col # 数値特徴量
##------------------------------------------------------------------------------

cols_cat = [c for c in test.columns if c not in cols_notcat] # 全カテゴリ特徴量
# ターゲットエンコ変数
target_encoded_cols = [c for c in test.columns if "target_" in c]
# ordinal cat変数
ordinal_cols_cat = [c for c in cols_cat if c not in target_encoded_cols]
"""
### ordinal_cols_catを削除
train = train.drop(ordinal_cols_cat, axis=1)
test = test.drop(ordinal_cols_cat, axis=1)
"""
cols_exp = [c for c in test.columns]

In [5]:
train

,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,MIS_Status,Sector,...,target_FranchiseCode,target_RevLineCr,target_LowDoc,target_Sector,target_ApprovalFY,target_city,target_State,target_BankState,target_UrbanRural,target_county_fips
0,163,21,1.0,0,0,1,1,3,1,23,...,0.919686,0.924868,0.908273,0.940289,0.876591,0.912698,0.908228,0.939937,0.945563,0.895935
1,84,6,1.0,4,0,0,0,3,1,19,...,0.880468,0.893256,0.907312,0.890526,0.927559,1.000000,0.906921,0.916435,0.946101,1.000000
2,242,45,1.0,4,90,0,1,3,1,7,...,0.882409,0.923958,0.908134,0.899727,0.898455,1.000000,0.949468,0.952703,0.825198,0.912693
3,237,4,1.0,0,0,0,1,3,1,6,...,0.881203,0.923889,0.907469,0.913000,0.903582,0.964435,0.927489,0.941023,0.944465,0.964435
4,184,0,1.0,0,0,0,1,3,1,23,...,0.880468,0.923693,0.907312,0.939921,0.920891,0.916309,0.883054,0.879085,0.946101,0.919112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42302,283,14,1.0,0,0,1,1,3,1,23,...,0.917544,0.923766,0.908344,0.940943,0.874442,0.957404,0.906085,0.883080,0.945499,0.957404
42303,53,2,1.0,0,0,0,4,3,1,7,...,0.882684,0.816008,0.908344,0.898218,0.826872,0.903226,0.879636,0.941487,0.825253,0.910778
42304,59,6,2.0,0,0,1,1,3,1,7,...,0.919606,0.923958,0.908134,0.899727,0.860706,0.928058,0.893253,0.875952,0.944626,0.934211
42305,295,18,1.0,0,8,0,1,3,1,7,...,0.881769,0.924868,0.908273,0.897770,0.917582,1.000000,0.808537,0.784226,0.945563,1.000000


In [6]:
def train_random_forest(train, cols_exp, col_target, params=None):

    if params is None:
        params = {}

    # クラス不均衡を考慮した重みの設定
    class_weight = {0: 9, 1: 1}  # ここでクラスの重みを調整

    params_add = {
        "class_weight": class_weight,
        # "random_state": 0
    }
    params |= params_add

    x = train[cols_exp].to_numpy()
    y = train[col_target].to_numpy()

    # K-fold
    kf = KFold(n_splits=5, shuffle=True, random_state=0)
    y_valid_pred_lst = []
    idx_valid_lst = []
    clf_lst = []

    # cross validation
    for fold, (idx_train, idx_valid) in enumerate(kf.split(x)):
        print("fold", fold)
        x_train = x[idx_train, :]
        x_valid = x[idx_valid, :]
        y_train = y[idx_train]
        y_valid = y[idx_valid]

        # modeling
        clf = RandomForestClassifier(random_state=0).fit(x_train, y_train)

        # oof
        y_valid_pred = clf.predict_proba(x_valid)
        y_valid_pred_lst.append(y_valid_pred)
        idx_valid_lst.append(idx_valid)
        clf_lst.append(clf)

    idx_valid = np.hstack(idx_valid_lst)
    y_valid_pred = np.vstack(y_valid_pred_lst)
    oof_pred = y_valid_pred[np.argsort(idx_valid)]

    return clf_lst, oof_pred

In [7]:
def predict_test(x_test, clf_lst):
    y_test_pred_lst = []

    for clf in clf_lst:
        y_test_pred = clf.predict_proba(x_test)
        y_test_pred_lst.append(y_test_pred)

    y_test_pred = np.mean(y_test_pred_lst, axis=0)
    return y_test_pred

In [8]:
col_target = "MIS_Status"
print("col_target =", col_target, "-"*50)

# train random forest
clf_lst, oof_pred = train_random_forest(train, cols_exp, col_target)

# predict test with CV ensemble
y_test_pred = predict_test(test[cols_exp].to_numpy(), clf_lst)

# record
oof_pred_df = pd.DataFrame(oof_pred, columns=[f"MIS_Status_{h}" for h in range(2)])
test_pred_df = pd.DataFrame(y_test_pred, columns=[f"MIS_Status_{h}" for h in range(2)])

col_target = MIS_Status --------------------------------------------------
fold 0
fold 1
fold 2
fold 3
fold 4


In [9]:
from sklearn.metrics import f1_score

# 実際のターゲット値
y_true = train[col_target].values

# OOF予測値をクラスに変換（確率から最も高いクラスを選択）
# 二値分類の場合、0.5を閾値としてクラスを決定する
y_oof_pred_class = (oof_pred[:, 1] > 0.5).astype(int)

# mean F1スコア（マクロ平均）を計算
mean_f1_score_macro = f1_score(y_true, y_oof_pred_class, average='macro')
print(f'Mean F1 Score (Macro): {mean_f1_score_macro}')


Mean F1 Score (Macro): 0.6369545959406855


In [10]:
# save
oof_pred_df.to_csv(f"/content/drive/MyDrive/signate/債務不履行/model_feat/oof_pred_randomforest_{feat}.csv", index=False)
test_pred_df.to_csv(f"/content/drive/MyDrive/signate/債務不履行/model_feat/test_pred_randomforest_{feat}.csv", index=False)